<a href="https://colab.research.google.com/github/earendil94/SMLExam/blob/master/Leo_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**STATISTICAL MACHINE LEARNING**

ARRIGHI Leonardo, BRAND Francesco, DORIGO Claudia


Dataset folder is saved in "/content/drive/My Drive/SML/SML_Project".

In [2]:
# link colab and drive
from google.colab import drive
drive.mount("/content/drive", force_remount=True)
# then follow passages

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
import numpy as np

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
import pandas as pd

import torchvision
from torchvision import transforms
from IPython import display

from PIL import Image
import glob

torch.manual_seed(160898)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Device: {}'.format(device))

Device: cuda:0


In [ ]:
csv = '/content/drive/My Drive/SML/SML_Project/Chunk1/results.csv'
imgs = '/content/drive/My\ Drive/SML/SML_Project/Chunk1/img1_200/'

!ls {imgs} | wc

    200     200    2382


In [ ]:
# Load csv
df = pd.read_csv(csv, delimiter='|') 
print(df.shape)
print(df.columns[2], df.columns[2] == ' comment') 
df[' comment'].values[0]
df.head(6)

(158915, 3)
 comment True


,image_name,comment_number,comment
0,1000092795.jpg,0,Two young guys with shaggy hair look at their...
1,1000092795.jpg,1,"Two young , White males are outside near many..."
2,1000092795.jpg,2,Two men in green shirts are standing in a yard .
3,1000092795.jpg,3,A man in a blue shirt standing in a garden .
4,1000092795.jpg,4,Two friends enjoy time spent together .
5,10002456.jpg,0,Several men in hard hats are operating a gian...


## Preprocessing Captions

In [4]:
#To put into a module
import os
from torchtext.data import Field
from torchtext.data import TabularDataset
from torchtext.data import Iterator

def prepare_data(path, input_file, output_file):
  input_path = os.path.join(path, input_file)
  output_path = os.path.join(path, output_file)
  df = pd.read_csv(input_path, sep = "|")
  captions_array = df[' comment']
  captions_array.fillna("", inplace=True)
  
  image_names = df["image_name"].values
  image_number = []
  for i in range(0, len(image_names)):
    image_number.append(image_names[i].split('.')[0])

  df.drop(labels=['image_name', ' comment_number'], axis=1, inplace=True)
  df.index = image_number
  df.to_csv(output_path, index_label="image_number")
  return output_path

def build_vocab(path_to_caption_file, caption_file):
  output_path = prepare_data(path_to_caption_file, caption_file, "clean.csv")
  tokenize = lambda x : x.split()
  TEXT = Field(sequential = True, tokenize = tokenize, lower=True, init_token='<start>', eos_token='<end>')
  LABEL = Field(sequential=False, use_vocab=False)
  td_datafields = [("image_number", LABEL ),
                  ("comment", TEXT)]

  trn = TabularDataset(
              path=output_path, # the root directory where the data lies
              format='csv',
              skip_header=True, # if your csv header has a header, make sure to pass this to ensure it doesn't get proceesed as data!
              fields=td_datafields,
              )
  
  TEXT.build_vocab(trn)
  return TEXT.vocab

def word_caption_to_index(path_to_caption_file, caption_file):
  '''
    This function takes as input the file containing the captions
    and returns a matrix of the captions indexed with respect to the inner
    vocabulary, as well as an array that can be used to map the indexed caption
    to the image it belongs to.
    @path_to_caption_file: the path to file containing the captions
    @caption_file: the name of the file containing the captions
  '''

  output_path = prepare_data(path_to_caption_file, caption_file, "clean.csv")
  tokenize = lambda x : x.split()
  TEXT = Field(sequential = True, tokenize = tokenize, lower=True, init_token='<start>', eos_token='<end>')
  LABEL = Field(sequential=False, use_vocab=False)

  td_datafields = [("image_number", LABEL ),
                  ("comment", TEXT)]

  trn = TabularDataset(
                path=output_path, # the root directory where the data lies
                format='csv',
                skip_header=True, # if your csv header has a header, make sure to pass this to ensure it doesn't get proceesed as data!
                fields=td_datafields
                )
  
  TEXT.build_vocab(trn)
  train_iter = Iterator(trn, batch_size=len(trn), device = -1)

  for i in train_iter: #TODO:I really havent got it how this still works honestly
    cmt = i.comment
    img = i.image_number

  return cmt.T, img

def vocab_as_dict(path_to_caption_file, caption_file):
  vocab = build_vocab(path_to_caption_file, caption_file)
  return vocab.stoi

def tensor_to_word(indexed_word, vocab):
  for i in indexed_word:
    k = i.item()
    if k == 1:
      break
    else:
      print(vocab.itos[k], end = " ")
    
  

def get_caption_from_image(caption_indexes, caption_refs, image_number):
  #If image name actually has the .jpg tail
  caption_refs = (caption_refs == int(image_number))
  caption_refs = caption_refs.nonzero().T.numpy()[0].tolist()
  return caption_indexes[caption_refs]

def buildCaptionDict(path_to_caption_file, caption_file):

  caption_index, refs = word_caption_to_index(path_to_caption_file, caption_file)
  refs_list = refs.numpy().tolist()
  refs_set = set(refs_list)
  unique_refs_list = list(refs_set)

  df = []
  df = pd.DataFrame(columns=["Image_number", "Caption_1", "Caption_2", "Caption_3", "Caption_4", "Caption_5"])

  for i in range(0, len(unique_refs_list)):
    capt_1 = get_caption_from_image(caption_index, refs, unique_refs_list[i])[0]
    capt_2 = get_caption_from_image(caption_index, refs, unique_refs_list[i])[1]
    capt_3 = get_caption_from_image(caption_index, refs, unique_refs_list[i])[2]
    capt_4 = get_caption_from_image(caption_index, refs, unique_refs_list[i])[3]
    capt_5 = get_caption_from_image(caption_index, refs, unique_refs_list[i])[4]
    df.loc[i] = [unique_refs_list[i], capt_1, capt_2, capt_3, capt_4, capt_5]

  df.set_index('Image_number', inplace=True)
  capt_dict = df.to_dict('index')

  return capt_dict


In [ ]:
# Load images
# needed transformation: to tensor and normalize
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

input_images = []
for filename in glob.glob("/content/drive/My Drive/SML/SML_Project/Chunk1/img1_200/*.jpg"):
    im=Image.open(filename)
    im=transform(im) # apply transformation while loading
    input_images.append(im)


In [ ]:
for i in range(10):
  print(input_images[i].shape)
# not all images have the same shape but it shouldn't be a problem for ResNet


torch.Size([3, 500, 335])
torch.Size([3, 375, 500])
torch.Size([3, 332, 500])
torch.Size([3, 450, 450])
torch.Size([3, 338, 450])
torch.Size([3, 500, 375])
torch.Size([3, 375, 500])
torch.Size([3, 374, 500])
torch.Size([3, 338, 450])
torch.Size([3, 333, 500])


In [ ]:
import torch.optim
import torch.utils.data
from torch import nn
import torchvision.transforms as transforms
from torch.nn.utils.rnn import pack_padded_sequence

# Model parameters
#decoder_dim = 
dropout = 0.5 # suggested

# Parameters
epoch = 1000
start_epoch = 0
epochs_since_improvement = 0 # keeps track of number of epochs since there's been an improvement in validation BLEU
batch_size = 32
decoder_lr = 1e-4  # learning rate
checkpoint = None
bleu4 = 0. # bleu score -> https://www.aclweb.org/anthology/P02-1040.pdf
workers = 1 # for dataloaders: how many subprocesses to use for data loading. 0 means that the data will be loaded in the main process

In [ ]:
# maybe this should be saved in a utils.py file

def adjust_learning_rate(optimizer, shrink_factor):
  for param_group in optimizer.param_groups:
    param_group['lr'] = param_group['lr'] * shrink_factor

def clip_gradient(optimizer, grad_clip):
  for i in optimizer.param_groups:
    for param in group['params']:
      if param.grad is not None:
        param.grad.data.clamp_(-grad_clip, grad_clip)



In [ ]:
# Vocabulary

# Decoder
#decoder = RNN(decoder_dim, vocabolary_size, dropout = dropout)

decoder_optimizer = torch.optim.Adam(params=filter(lambda p: p.requires_grad, 
                                                   decoder.parameters()),
                                     lr=decoder_lr)
decoder = decoder.to(device)

# Loss function
criterion = nn.CrossEntropyLoss().to(device)

# Epochs
for epoch in range(start_epoch, epochs):
  # Decay learning rate if there is no improvement for 8 consecutive epochs, and terminate training after 20
  if epochs_since_improvement == 20:
    break
  if epochs_since_improvement > 0 and epochs_since_improvement % 8 == 0:
    adjust_learning_rate(decoder_optimizer, 0.8)

  # Training
  train(train_loader=train_loader,
        decoder=decoder,
        criterion=criterion,
        decoder_optimizer=decoder_optimizer,
        epoch=epoch)
  
  # Validation
  #bleu4_new

  # BLEU4
  flag1 = bleu4_new > bleu4
  bleu4 = max(bleu4_new, bleu4)
  if not flag1:
    epochs_since_improvement += 1
  else:
    epochs_since_improvement = 0

NameError: ignored

In [ ]:
import torch
from torch.utils.data import Dataset
import pandas as pd

class Splitted_dataset(Dataset):

  def __init__(self, csv, folder):
    # csv: path of the csv file with captions
    # folder: directory with all images
    
    self.df = pd.read_csv(csv)
    self.captions_col = self.df['comment']
    self.image_name_col = self.df['image_name']

    self.df['length'] = self.captions_col.apply(lambda x: len(x.split()))
    self.length_col = self.df['length']

    self.folder = folder

  def __len__(self):
    return len(self.df)

  # not useful atm: def __getitem__(self, image_index)

In [69]:
import os
import numpy as np
import torch
import pandas as pd
import torchvision
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from collections import defaultdict
import random
from torch import tensor


def create (csv, folder):
  train_set = Splitted_dataset(csv = csv, 
                               folder = folder)
  return train_set

def split(csv, folder, val_size):
  '''
    @csv: name of the csv
    @folder: path to csv
  '''
  path_to_csv = os.path.join(folder, csv)
  output_path_train = os.path.join(folder, "train.csv")
  output_path_val = os.path.join(folder, "val.csv")

  df = pd.read_csv(path_to_csv, sep='|')

  # we want to split our dataset given itself and the % of sample for validation
  num = len(df)
  index = list(range(num))
  np.random.shuffle(index) # pick at random
  flag_split = int(val_size * num)

  train_index = index[flag_split:]
  validation_index = index[:flag_split]

  # https://pytorch.org/docs/stable/data.html -> Samples elements randomly from a given list of indices, without replacement
  #train_sampler = SubsetRandomSampler(train_index)
  #validation_sampler = SubsetRandomSampler(validation_index)

  df_train = df.iloc[train_index]
  df_val = df.iloc[validation_index]

  df_train.to_csv(output_path_train, index=False)
  df_val.to_csv(output_path_val, index=False)

  return df_train, df_val

def loaders(train_set, train_samplers, validation_sampler, batch_size, val_size, num_workers, csv, folder):

  train_sampler, validation_sampler = split(train_set, val_size)
  train_loader = DataLoader(train_set,
                            batch_size = batch_size,
                            sampler = train_sampler,
                            num_workers = num_workers)
  val_loader = DataLoader(train_set,
                          batch_size = batch_size,
                          sampler = validation_sampler,
                          num_workers = num_workers)
  return train_loader, val_loader

class imageCaptionDataset(Dataset):

  '''
    What we want to achieve is a mapping of type:
    { img_name: [img_tensor, caption_1, caption_2, caption_3, caption_4, caption_5]}
  '''

  #TODO: should have a function that puts the two items in the same dictionary
  def __init__(self, preProcessedImages, preProcessedCaptions):
    '''
      Here we should input two dictionaries preProcessedImages and preProcessedCaptions
    '''
    self.big_Dict = self.__merge_dict__(preProcessedImages, preProcessedCaptions)
    #self.processed = self.__getitem__(key) #TODO (Brand): I don't quite get this
    
  #TODO: should return the value associated with the given key
  def __getitem__(self, key):
    '''
      Given dictionary and key, it returns a tuple eith the value of the dictionary (best caption, image) and the length of the best caption 
    '''
    self.big_Dict = self.__one_pick__(self.big_Dict)
    val = []
    for i in big_Dict.get(key):
      val.append(i)
    val.append(len(val[0]))
    return val

  def __merge_dict__(self, d1, d2):
   '''
     Merge dictionaries and keep values of common keys in list
   '''
   d3 = {**d1, **d2}
   for key, value in d3.items():
       if key in d1 and key in d2:
               d3[key] = [value , d1[key]]
   return d3

   # this method will be changed
  def __choose_best__(self, d):
    '''
      Choose a particular caption
    '''
    val = random.choice(list(d.values()))
    return val

  # service function
  def __one_pick__(self, d):
    '''
      It returns a dictionary {best caption, image}
    '''
    tryd = {}
    for e in d:
      print(e)
      tryd[e] = {self.__choose_best__(d[e][0]), d[e][1]}
    return tryd

  #

#class imageCaptionSampler(Sampler):
  
 # def __iter__()


  

In [ ]:
def train(train_loader, decoder, criterion, decoder_optimizer, epoch):

  decoder.train() # train mode

  # Batch
  for i, (image, caption, captionlen) in enumerate(train_loader):
    
    # Move to GPU, if available
    image = image.to(device)
    caption = caption.to(device)
    captionlen = captionlen.to(device)

    # Forward 
    #decoder output = decoder(image, caption, captionlen)

    # Remove timesteps that we didn't decode at, or are pads
    # https://pytorch.org/docs/master/generated/torch.nn.utils.rnn.pack_padded_sequence.html
    # https://www.tensorflow.org/tutorials/text/image_captioning
    scores, _ = pack_padded_sequence(scores, decode_lengths, batch_first=True)
    targets, _ = pack_padded_sequence(targets, decode_lengths, batch_first=True)

    # Calculate loss
    loss = criterion(scores, targets)

    # Back propagation
    decoder_optimizer.zero_grad()
    loss.backward()

    # Clip gradients
    # https://machinelearningmastery.com/how-to-avoid-exploding-gradients-in-neural-networks-with-gradient-clipping/
    if grad_clip is not None:
      clip_gradient(decoder_optimizer, grad_clip)

    # Update weights
    decoder_optimizer.step()

    


In [ ]:
split('results.csv', '/content/drive/My Drive/SML_Project/Chunk1', 0.2)

(            image_name  ...                                            comment
 69824   3417672954.jpg  ...                       A skateboarder in the city .
 128769  4971283602.jpg  ...              Two students kissing at a crosswalk .
 59869   3194533056.jpg  ...                        Women are dancing outside .
 18745   2156131463.jpg  ...   People in bathing suits are on a beach lookin...
 126399  4934630252.jpg  ...               A person sitting on a hill outside .
 ...                ...  ...                                                ...
 150788  7376855592.jpg  ...     A rodeo player is being taken down by a bull .
 72501   3475861944.jpg  ...   A group of people in costumes are gathered , ...
 4328     128479788.jpg  ...   Two kids with blond-hair sit atop a four-whee...
 19200   2174648405.jpg  ...   Women in dresses work the assembly line while...
 53373   3039675864.jpg  ...      A man riding a motorcycle down the sidewalk .
 
 [127132 rows x 3 columns],
          

In [21]:
import shelve

def dict_from_shelve(shelve_file):
  dictionary = {}
  d=shelve.open(shelve_file)
  for k in d.keys():
    dictionary[k]=d[k]
  return dictionary


In [58]:
a = dict_from_shelve('/content/drive/My Drive/SML_Project/Chunk1/img_shelve')
b = buildCaptionDict('/content/drive/My Drive/SML_Project/Chunk1','results.csv')

The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.


In [70]:
a = {int(k): v for k,v in a.items()} #TODO:I have integers as keys, Cla has strings: This standardizes to int

c = imageCaptionDataset(a,b)
c.big_Dict[256063]
#c.__getitem__(256063) #TODO: still needs to be fixed

10002456
10287332
1243756
1317156
134206
1369162
2069887
2209317
256063
2868798
3367399
3494059
36979
438106
4413714
490870
5060753
5566972
5570254
6100315
6261030
6331511
6335241
6678735
6734417
81641
960092
10082347
10101477
148284
2317271
2760167
3012229
371897
4183120
4926723
5570219
5722658
5871970
6338733
6905083
6920532
7013217
7510394
7656601
8378599
8443156
8664922
9324151
9600569
9637989
10459869
1254659
2148982
2784746
3160699
3662865
371902
3787801
3996401
4414061
4906946
5867606
6214447
6901479
7300628
7598674
7598946
10010052
205842
2689611
3219606
371903
4515460
4985704
5230968
5400154
5558592
5648321
6338704
6398988
6659698
6664030
667626
675153
6827028
7015055
807129
8404753
854749
8664920
8680922
10404007
1624481
353913
390369
4376178
4489731
4576671
4749855
5444724
5624522
574181
5771732
5791244
5958182
7527111
8063007
8849890
8859482
9035232
9556225
9950858
10188041
10350842
178045
2192573
3043766
3537322
3680138
4429660
5629300
5791568
5914327
5918675
6054169
61551

KeyError: ignored